In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import xgboost as xg

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier, Pool

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
DF_train = pd.read_csv('/kaggle/input/water-pollution/water pollution (data for classification models)/Train.csv', delimiter = ',')
DF_test = pd.read_csv('/kaggle/input/water-pollution/water pollution (data for classification models)/Test.csv', delimiter = ',')
Target = pd.read_csv('/kaggle/input/water-pollution/water pollution (data for classification models)/Target.csv', delimiter = ',')
Submission = pd.read_csv('/kaggle/input/water-pollution/water pollution (data for classification models)/Submission.csv', delimiter = ',')

In [ ]:
Target.describe()

### Concatinate train and test

In [ ]:
df = pd.concat([DF_train, DF_test])

In [ ]:
df.info()

## View categorical and numerical columns


In [ ]:
cat_columns = []
num_columns = []

for column_name in df.columns:
    if (df[column_name].dtypes == object):
        cat_columns +=[column_name]
    else:
        num_columns +=[column_name]

print('categorical columns:\t ',cat_columns, '\n len = ',len(cat_columns))

print('numerical columns:\t ',  num_columns, '\n len = ',len(num_columns))

### Move 'year' to num columns


In [ ]:
cat_columns = ['code', 'period', 'id', 'Country', 'year']
num_columns = ['Unnamed: 0', 'tourists', 'venue', 'rate', 'food', 'glass', 'metal', 'other', 'paper', 'plastic', 'leather', 'green_waste', 'waste_recycling'] 

# OHE

In [ ]:
df_ohe = pd.get_dummies(df[cat_columns])

In [ ]:
df_all = pd.concat([df_ohe, df[num_columns]], axis=1)

# Train test split

In [ ]:
# Creating StandardScaler Object
scaler = StandardScaler()
df_scale = pd.DataFrame(scaler.fit_transform(df_all[num_columns]))
df_all = pd.concat([df_ohe.reset_index(drop=True), df_scale.reset_index(drop=True)], axis=1)

In [ ]:
len_test = len(DF_test)
train = df_all.iloc[:-len_test]
test = df_all.iloc[-len_test:]
print(DF_train.shape[0], train.shape[0], DF_test.shape[0], test.shape[0])

In [ ]:
#X, y = pd.DataFrame(train).values, Target['polution_clf'].values
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#X_train.shape[0], X_test.shape[0], y_train.shape[0], y_test.shape[0]

## Preprocessing 

In [ ]:
from sklearn.preprocessing import PowerTransformer

from sklearn.impute import SimpleImputer

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, OrdinalEncoder

In [ ]:
X_train = DF_train.copy()
X_test = DF_test.copy()
y_train = Target['polution_clf']

In [ ]:
X_train_, X_val, y_train_, y_val = train_test_split(X_train,y_train,test_size = 0.1, random_state = 34)

### Model scoring function

In [ ]:
def calculate_f1_score(model_pipe, X, y):
    """F1 calculation. 
    
    Parameters:
    ===========
    model_pipe: model or pipeline
    X: features
    y: real values
    """
    y_model = model_pipe.predict(X)
    return f1_score(y, y_model)

# Data preparation pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

Numerical data preprocessing (insert missing values and normalization) 

In [ ]:
numerical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
   ('scaler', MinMaxScaler())
])
numerical_pipe

Categorical data preprocessing (insert missing values + OHE)

In [ ]:
categorical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent', )),
    ('encoder', OneHotEncoder(drop='if_binary', handle_unknown='ignore', sparse=False))
])
categorical_pipe

### Concatinate preprocessing pipilines

In [ ]:
preprocessors = ColumnTransformer(transformers=[
    ('num', numerical_pipe, num_columns[1:]),
    ('cat', categorical_pipe, cat_columns)
])
preprocessors

### Concatinate preprocessing pipeline and model

In [ ]:
pipe = Pipeline([
    ('preprocessors', preprocessors),
    ('model', LogisticRegression(C=2, random_state=42, max_iter = 100000))
])

In [ ]:
pipe.fit(X_train_, y_train_)

# LogisticRegression

In [ ]:
print (classification_report(y_test, y_predict, target_names=['polution_clf']))

# XGBoost

# CatBoost

## Sumbission results

In [ ]:
! rm /kaggle/working/water-pollution/Submission.csv
! mkdir /kaggle/working/water-pollution/

In [ ]:
Submission.to_csv('/kaggle/input/water-pollution/water pollution (data for classification models)/Submission.csv',index=False)

In [ ]:
! mkdir ~/.kaggle
! touch ~/.kaggle/kaggle.json
! echo '{"username":"hydramst","key": "" }' > ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
cat ~/.kaggle/kaggle.json

In [ ]:
#! kaggle competitions submit -c sf-matml-2022-classification -f Submission.csv -m "Message"